In [5]:
%pip install -U setuptools==70.0.0
%pip install -U wikipedia boto3 llama-index-llms-bedrock llama-index-graph-stores-neptune llama-index-embeddings-bedrock llama-index-readers-file


  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
Note: you may need to restart the kernel to use updated packages.
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.4 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=f25752439df1ae839d57fdeb69196d8f301bd56fbd68aede18030d40d2447c17
  S

In [6]:
import wikipedia
import os

wikipedia.set_lang("ja")
state = "ドラゴンボール"
directory = "./data"
short_summary = False

os.makedirs(directory, exist_ok=True)
try:
    title = wikipedia.page(state).title.lower().replace(" ", "_")
    content = (wikipedia.page(state).summary if short_summary else wikipedia.page(state).content)
    content = content.strip()
    filename = os.path.join(directory, f"{title}.txt")
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"Saving {filename}")
except Exception:
    print("Error")

Saving ./data/ドラゴンボール.txt


In [29]:
import os

from llama_index.graph_stores.neptune import NeptuneDatabaseGraphStore
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import (
    StorageContext,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    Settings,
)
from IPython.display import Markdown, display
import boto3

session = boto3.Session()
config = boto3.session.Config(region_name='us-west-2')
client = session.client('bedrock-runtime', config=config)

llm = Bedrock(
	model="anthropic.claude-3-sonnet-20240229-v1:0",
)

embed_model = BedrockEmbedding(
    model="amazon.titan-embed-text-v1",
)

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [30]:
documents = SimpleDirectoryReader(input_dir="./data").load_data()

In [31]:
graph_store = NeptuneDatabaseGraphStore(
    host="tf-20241108043339787000000001.c7gmsaquakio.us-west-2.neptune.amazonaws.com",
    port=8182
)

In [32]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: You don't have access to the model with the specified model ID.

In [ ]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# そこそこ時間かかる
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
)

In [ ]:
index.storage_context.persist("./persist")

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "悟空に友達とお嫁さんはいるの？"
)
display(Markdown(f"{response}"))

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "悟空がカメハウスで共に暮らしていたのは誰？"
)
display(Markdown(f"{response}"))